In [10]:
import pandas as pd
import numpy as np
from pathlib import Path
from typing import List, Dict, Tuple
from dataclasses import dataclass

aircraft_speeds = {}

@dataclass
class Cell4D:
    """Represents a 4D airspace cell"""
    x_min: float  
    x_max: float
    y_min: float  
    y_max: float
    z_min: float  
    z_max: float
    t_min: int    
    t_max: int
    aircraft_records: List[Dict]

class ComplexityCalculator:
    def __init__(self, 
                 cell_size_nm: float = 20.0,
                 cell_height_ft: float = 3000.0,
                 time_step_min: float = 10.0):
        
        self.cell_size_nm = cell_size_nm
        self.cell_height_ft = cell_height_ft
        self.time_step_sec = time_step_min * 1
        self.cells: Dict[Tuple, Cell4D] = {}
        
    def _determine_vertical_state(self, alt1: float, alt2: float, time_diff: float) -> str:
        """Determine if aircraft is climbing, cruising, or descending"""
        VERTICAL_RATE_THRESHOLD = 500.0  # ft/min
        
        # Convert threshold to per second
        threshold_per_sec = VERTICAL_RATE_THRESHOLD / 60 
        alt_diff = alt2 - alt1
        rate = alt_diff / time_diff if time_diff > 0 else 0
        
        if abs(rate) < threshold_per_sec:
            return "cruise"
        return "climb" if rate > 0 else "descent"
    
    def _check_temporal_overlap(self, ac1: Dict, ac2: Dict) -> float:
        """
        Calculate the duration of temporal overlap between two aircraft in a cell
        Returns overlap duration in hours
        """
        start1, end1 = ac1['entry_time'], ac1['exit_time']
        start2, end2 = ac2['entry_time'], ac2['exit_time']
        
        # Calculate overlap
        overlap_start = max(start1, start2)
        overlap_end = min(end1, end2)
        
        if overlap_end > overlap_start:
            return (overlap_end - overlap_start) / 3600  # Convert to hours
        return 0
    
    
    def _extract_aircraft_type(self, flight_id: str) -> str:
        """Extract aircraft type from flight ID (e.g., 'SQ10_B738_trajectory' -> 'B738')"""
        parts = flight_id.split('_')
        if len(parts) > 1:
            return parts[1]
        return ''

    def create_4d_grid(self, all_data: pd.DataFrame) -> None:
        """Create 4D grid and assign aircraft to cells"""
        # Find bounds of the airspace
        lat_min, lat_max = all_data['latitude'].min(), all_data['latitude'].max()
        lon_min, lon_max = all_data['longitude'].min(), all_data['longitude'].max()
        alt_min, alt_max = all_data['altitude'].min(), all_data['altitude'].max()
        time_min, time_max = all_data['timestamp'].min(), all_data['timestamp'].max()

        # Create grid edges
        lat_edges = np.arange(lat_min, lat_max + self.cell_size_nm/60, self.cell_size_nm/60)
        lon_edges = np.arange(lon_min, lon_max + self.cell_size_nm/60, self.cell_size_nm/60)
        alt_edges = np.arange(alt_min, alt_max + self.cell_height_ft, self.cell_height_ft)
        time_edges = np.arange(time_min, time_max + self.time_step_sec, self.time_step_sec)

        # Group data by flight_id to process each trajectory
        for flight_id, flight_data in all_data.groupby('flight_id'):
            flight_data = flight_data.sort_values('timestamp')
            aircraft_type = self._extract_aircraft_type(flight_id)
            
            for idx in range(len(flight_data)-1):
                row = flight_data.iloc[idx]
                next_row = flight_data.iloc[idx + 1]
                
                # Find cell indices
                cell_x = np.digitize(row['longitude'], lon_edges) - 1
                cell_y = np.digitize(row['latitude'], lat_edges) - 1
                cell_z = np.digitize(row['altitude'], alt_edges) - 1
                cell_t = np.digitize(row['timestamp'], time_edges) - 1
                
                cell_key = (cell_x, cell_y, cell_z, cell_t)
                
                if cell_key not in self.cells:
                    self.cells[cell_key] = Cell4D(
                        x_min=lon_edges[cell_x], x_max=lon_edges[cell_x + 1],
                        y_min=lat_edges[cell_y], y_max=lat_edges[cell_y + 1],
                        z_min=alt_edges[cell_z], z_max=alt_edges[cell_z + 1],
                        t_min=time_edges[cell_t], t_max=time_edges[cell_t + 1],
                        aircraft_records=[]
                    )
                
                time_diff = next_row['timestamp'] - row['timestamp']
                
                # Store aircraft record with entry and exit times
                self.cells[cell_key].aircraft_records.append({
                    'flight_id': flight_id,
                    'aircraft_type': aircraft_type,
                    'entry_time': row['timestamp'],
                    'exit_time': min(next_row['timestamp'], time_edges[cell_t + 1]),
                    'altitude': row['altitude'],
                    'waypoint_segment': row['waypoint_segment'],
                    'heading': np.arctan2(next_row['longitude'] - row['longitude'],
                                        next_row['latitude'] - row['latitude']),
                    'vertical_state': self._determine_vertical_state(
                        row['altitude'], 
                        next_row['altitude'],
                        time_diff
                    )
                })

    def calculate_complexity_metrics(self) -> Dict:
        """Calculate complexity metrics for the airspace"""
        total_flight_hours = 0
        total_interaction_hours = 0
        vertical_interaction_hours = 0
        horizontal_interaction_hours = 0
        speed_interaction_hours = 0
        
        for cell in self.cells.values():
            n_aircraft = len(cell.aircraft_records)
            if n_aircraft < 2:
                if n_aircraft == 1:
                    ac = cell.aircraft_records[0]
                    time_in_cell = (ac['exit_time'] - ac['entry_time']) / 3600
                    total_flight_hours += time_in_cell
                continue
            
            # Calculate interactions between aircraft pairs in the cell
            for i in range(n_aircraft):
                ac1 = cell.aircraft_records[i]
                time_in_cell = (ac1['exit_time'] - ac1['entry_time']) / 3600
                total_flight_hours += time_in_cell
                
                for j in range(i + 1, n_aircraft):
                    ac2 = cell.aircraft_records[j]
                    
                    # Calculate actual temporal overlap
                    interaction_time = self._check_temporal_overlap(ac1, ac2)
                    
                    if interaction_time > 0:
                        total_interaction_hours += interaction_time
                        
                        # Vertical interactions
                        if ac1['vertical_state'] != ac2['vertical_state']:
                            vertical_interaction_hours += interaction_time
                        
                        # Horizontal interactions (heading difference > 20 degrees)
                        heading_diff = abs(ac1['heading'] - ac2['heading'])
                        heading_diff = min(heading_diff, 2*np.pi - heading_diff)
                        if heading_diff > np.radians(20):
                            horizontal_interaction_hours += interaction_time
                     
                        # Speed interactions (different aircraft types)
                        if ac1['aircraft_type'] != ac2['aircraft_type']:
                            speed_interaction_hours += interaction_time   
                        # Speed interactions (different vertical states or altitude separation)
                        # if (ac1['vertical_state'] != ac2['vertical_state'] or 
                        #     abs(ac1['altitude'] - ac2['altitude']) > 1000):
                        #     speed_interaction_hours += interaction_time
                            
            # for i in range(n_aircraft):
            #     ac1 = cell.aircraft_records[i]
            #     time_in_cell = (ac1['exit_time'] - ac1['entry_time']) / 3600
            #     total_flight_hours += time_in_cell
                
            #     for j in range(i + 1, n_aircraft):
            #         ac2 = cell.aircraft_records[j]
                    
            #         # Calculate interaction duration
            #         interaction_time = self._check_temporal_overlap(ac1, ac2)
                    
            #         if interaction_time > 0:
            #             total_interaction_hours += interaction_time
                    
            #         # Vertical interactions
            #         if ac1['vertical_state'] != ac2['vertical_state']:
            #             vertical_interaction_hours += interaction_time
                    
            #         # Horizontal interactions (different waypoint segments)
            #         if ac1['waypoint_segment'] != ac2['waypoint_segment']:
            #             horizontal_interaction_hours += interaction_time
                    
            #         # Speed interactions (based on altitude difference)
            #         if abs(ac1['altitude'] - ac2['altitude']) > 1000:
            #             speed_interaction_hours += interaction_time
        
        # Calculate metrics
        adjusted_density = total_interaction_hours / total_flight_hours if total_flight_hours > 0 else 0
        
        # Calculate relative indicators
        r_vdif = vertical_interaction_hours / total_interaction_hours if total_interaction_hours > 0 else 0
        r_hdif = horizontal_interaction_hours / total_interaction_hours if total_interaction_hours > 0 else 0
        r_sdif = speed_interaction_hours / total_interaction_hours if total_interaction_hours > 0 else 0
        
        # Calculate structural index and complexity score
        structural_index = r_vdif + r_hdif + r_sdif
        complexity_score = adjusted_density * structural_index
        
        return {
            'adjusted_density': adjusted_density,
            'vertical_interactions_per_flight_hour': vertical_interaction_hours / total_flight_hours,
            'horizontal_interactions_per_flight_hour': horizontal_interaction_hours / total_flight_hours,
            'speed_interactions_per_flight_hour': speed_interaction_hours / total_flight_hours,
            'r_vdif': r_vdif,
            'r_hdif': r_hdif,
            'r_sdif': r_sdif,
            'structural_index': structural_index,
            'complexity_score': complexity_score,
            'total_flight_hours': total_flight_hours,
            'number_of_cells': len(self.cells)
        }


def calculate_airspace_complexity(trajectories_folder: str = 'aircraft_trajectories_24jan') -> Dict:
    """
    Calculate airspace complexity from all trajectory files in the specified folder
    """
    # Get all trajectory files from the folder
    trajectory_path = Path(trajectories_folder)
    trajectory_files = list(trajectory_path.glob("SQ*.csv"))
    
    if not trajectory_files:
        raise ValueError(f"No trajectory files found in {trajectories_folder}")
    
    print(f"Found {len(trajectory_files)} trajectory files")
    
    # Read all trajectories into a single DataFrame
    all_data = []
    for file in trajectory_files:
        print(f"Reading {file.name}")
        df = pd.read_csv(file)
        df['flight_id'] = file.stem  # Add flight ID from filename
        all_data.append(df)
    # print(all_data.grouby('flight_id'))
    # Combine all trajectories
    all_trajectories = pd.concat(all_data, ignore_index=True)
    print(f"Total data points: {len(all_trajectories)}")
    
    # Initialize calculator and process data
    calculator = ComplexityCalculator()
    calculator.create_4d_grid(all_trajectories)
    metrics = calculator.calculate_complexity_metrics()
    
    return metrics


In [6]:
trajectory_path = Path('aircraft_trajectories')
trajectory_files = list(trajectory_path.glob("SQ*.csv"))
all_data = []
for file in trajectory_files:
    print(f"Reading {file.name}")
    df = pd.read_csv(file)
    df['flight_id'] = file.stem  # Add flight ID from filename
    all_data.append(df)
    
all_trajectories = pd.concat(all_data, ignore_index=True)
c = ComplexityCalculator()
c.create_4d_grid(all_trajectories)

Reading SQ17_B738_trajectory.csv
Reading SQ115_B738_trajectory.csv
Reading SQ14_B734_trajectory.csv
Reading SQ114_B738_trajectory.csv
Reading SQ19_B744_trajectory.csv
Reading SQ111_B744_trajectory.csv
Reading SQ112_B744_trajectory.csv
Reading SQ13_B734_trajectory.csv
Reading SQ18_B738_trajectory.csv
Reading SQ116_B738_trajectory.csv
Reading SQ15_B738_trajectory.csv
Reading SQ11_B738_trajectory.csv
Reading SQ113_B738_trajectory.csv
Reading SQ16_B738_trajectory.csv
Reading SQ12_B734_trajectory.csv
Reading SQ10_B738_trajectory.csv
Reading SQ110_B744_trajectory.csv


In [14]:
type(c.cells)

dict

In [17]:
first_key, first_value = next(iter(c.cells.items()))
print(f"First key: {first_key}, First value: {first_value}")

First key: (np.int64(3), np.int64(0), np.int64(7), np.int64(0)), First value: Cell4D(x_min=np.float64(103.67194399999998), x_max=np.float64(104.00527733333331), y_min=np.float64(2.161111), y_max=np.float64(2.4944443333333335), z_min=np.float64(37333.330000000016), z_max=np.float64(40333.330000000016), t_min=np.float64(0.0), t_max=np.float64(3600.0), aircraft_records=[{'flight_id': 'SQ10_B738_trajectory', 'timestamp': np.int64(300), 'altitude': np.float64(39637.0), 'waypoint_segment': 'VMR-LENDA', 'vertical_state': 'descent', 'time_in_cell': np.float64(3300.0)}, {'flight_id': 'SQ10_B738_trajectory', 'timestamp': np.int64(310), 'altitude': np.float64(39274.0), 'waypoint_segment': 'VMR-LENDA', 'vertical_state': 'descent', 'time_in_cell': np.float64(3290.0)}, {'flight_id': 'SQ10_B738_trajectory', 'timestamp': np.int64(320), 'altitude': np.float64(38911.0), 'waypoint_segment': 'VMR-LENDA', 'vertical_state': 'descent', 'time_in_cell': np.float64(3280.0)}, {'flight_id': 'SQ10_B738_trajectory'

In [21]:
c.cells[np.int64(3), np.int64(0), np.int64(7), np.int64(0)].aircraft_records

[{'flight_id': 'SQ10_B738_trajectory',
  'timestamp': np.int64(300),
  'altitude': np.float64(39637.0),
  'waypoint_segment': 'VMR-LENDA',
  'vertical_state': 'descent',
  'time_in_cell': np.float64(3300.0)},
 {'flight_id': 'SQ10_B738_trajectory',
  'timestamp': np.int64(310),
  'altitude': np.float64(39274.0),
  'waypoint_segment': 'VMR-LENDA',
  'vertical_state': 'descent',
  'time_in_cell': np.float64(3290.0)},
 {'flight_id': 'SQ10_B738_trajectory',
  'timestamp': np.int64(320),
  'altitude': np.float64(38911.0),
  'waypoint_segment': 'VMR-LENDA',
  'vertical_state': 'descent',
  'time_in_cell': np.float64(3280.0)},
 {'flight_id': 'SQ10_B738_trajectory',
  'timestamp': np.int64(330),
  'altitude': np.float64(38548.0),
  'waypoint_segment': 'VMR-LENDA',
  'vertical_state': 'descent',
  'time_in_cell': np.float64(3270.0)},
 {'flight_id': 'SQ10_B738_trajectory',
  'timestamp': np.int64(340),
  'altitude': np.float64(38185.0),
  'waypoint_segment': 'VMR-LENDA',
  'vertical_state': 'desc

In [16]:
for k, v in c.cells.items():
    print(f"key: {k}, Value: {v}")

key: (np.int64(3), np.int64(0), np.int64(7), np.int64(0)), Value: Cell4D(x_min=np.float64(103.67194399999998), x_max=np.float64(104.00527733333331), y_min=np.float64(2.161111), y_max=np.float64(2.4944443333333335), z_min=np.float64(37333.330000000016), z_max=np.float64(40333.330000000016), t_min=np.float64(0.0), t_max=np.float64(3600.0), aircraft_records=[{'flight_id': 'SQ10_B738_trajectory', 'timestamp': np.int64(300), 'altitude': np.float64(39637.0), 'waypoint_segment': 'VMR-LENDA', 'vertical_state': 'descent', 'time_in_cell': np.float64(3300.0)}, {'flight_id': 'SQ10_B738_trajectory', 'timestamp': np.int64(310), 'altitude': np.float64(39274.0), 'waypoint_segment': 'VMR-LENDA', 'vertical_state': 'descent', 'time_in_cell': np.float64(3290.0)}, {'flight_id': 'SQ10_B738_trajectory', 'timestamp': np.int64(320), 'altitude': np.float64(38911.0), 'waypoint_segment': 'VMR-LENDA', 'vertical_state': 'descent', 'time_in_cell': np.float64(3280.0)}, {'flight_id': 'SQ10_B738_trajectory', 'timestamp

In [42]:
def calculate_airspace_complexity(trajectories_folder: str = 'aircraft_trajectories') -> Dict:
    """
    Calculate airspace complexity from all trajectory files in the specified folder
    """
    # Get all trajectory files from the folder
    trajectory_path = Path(trajectories_folder)
    trajectory_files = list(trajectory_path.glob("SQ*.csv"))
    
    if not trajectory_files:
        raise ValueError(f"No trajectory files found in {trajectories_folder}")
    
    print(f"Found {len(trajectory_files)} trajectory files")
    
    # Read all trajectories into a single DataFrame
    all_data = []
    for file in trajectory_files:
        print(f"Reading {file.name}")
        df = pd.read_csv(file)
        df['flight_id'] = file.stem  # Add flight ID from filename
        all_data.append(df)
    print(all_data.grouby('flight_id'))
    # Combine all trajectories
    all_trajectories = pd.concat(all_data, ignore_index=True)
    print(f"Total data points: {len(all_trajectories)}")
    
    # Initialize calculator and process data
    calculator = ComplexityCalculator()
    calculator.create_4d_grid(all_trajectories)
    metrics = calculator.calculate_complexity_metrics()
    
    return metrics

In [11]:
metrics = calculate_airspace_complexity()
metrics

Found 21 trajectory files
Reading SQ17_B738_trajectory.csv
Reading SQ111_A320_trajectory.csv
Reading SQ115_B738_trajectory.csv
Reading SQ112_A320_trajectory.csv
Reading SQ118_B734_trajectory.csv
Reading SQ14_A320_trajectory.csv
Reading SQ119_B734_trajectory.csv
Reading SQ110_A320_trajectory.csv
Reading SQ114_B738_trajectory.csv
Reading SQ117_B734_trajectory.csv
Reading SQ13_A320_trajectory.csv
Reading SQ18_B738_trajectory.csv
Reading SQ15_A320_trajectory.csv
Reading SQ11_B738_trajectory.csv
Reading SQ120_B734_trajectory.csv
Reading SQ113_B738_trajectory.csv
Reading SQ16_B738_trajectory.csv
Reading SQ19_B738_trajectory.csv
Reading SQ116_B734_trajectory.csv
Reading SQ10_B738_trajectory.csv
Reading SQ12_B738_trajectory.csv
Total data points: 4347


{'adjusted_density': 0.16967175219600603,
 'vertical_interactions_per_flight_hour': 0.006703652334719604,
 'horizontal_interactions_per_flight_hour': 0.06310679611649817,
 'speed_interactions_per_flight_hour': 0.04137771613499332,
 'r_vdif': 0.03950953678474126,
 'r_hdif': 0.37193460490463226,
 'r_sdif': 0.24386920980926444,
 'structural_index': 0.655313351498638,
 'complexity_score': 0.1111881645862111,
 'total_flight_hours': 12.016666666667906,
 'number_of_cells': 3640}

In [9]:
metrics = calculate_airspace_complexity()
metrics

Found 19 trajectory files
Reading SQ17_B738_trajectory.csv
Reading SQ111_A320_trajectory.csv
Reading SQ115_B738_trajectory.csv
Reading SQ112_A320_trajectory.csv
Reading SQ118_B734_trajectory.csv
Reading SQ14_A320_trajectory.csv
Reading SQ110_A320_trajectory.csv
Reading SQ114_B738_trajectory.csv
Reading SQ117_B734_trajectory.csv
Reading SQ13_A320_trajectory.csv
Reading SQ18_B738_trajectory.csv
Reading SQ15_A320_trajectory.csv
Reading SQ11_B738_trajectory.csv
Reading SQ113_B738_trajectory.csv
Reading SQ16_B738_trajectory.csv
Reading SQ19_B738_trajectory.csv
Reading SQ116_B734_trajectory.csv
Reading SQ10_B738_trajectory.csv
Reading SQ12_B738_trajectory.csv
Total data points: 3987


{'adjusted_density': 0.17893145161288507,
 'vertical_interactions_per_flight_hour': 0.0017641129032256331,
 'horizontal_interactions_per_flight_hour': 0.07459677419354084,
 'speed_interactions_per_flight_hour': 0.05141129032257546,
 'r_vdif': 0.009859154929577497,
 'r_hdif': 0.4169014084507044,
 'r_sdif': 0.287323943661972,
 'structural_index': 0.7140845070422539,
 'complexity_score': 0.12777217741934194,
 'total_flight_hours': 11.022222222223306,
 'number_of_cells': 3305}

In [ ]:
trajectory_path = Path('aircraft_trajectories')
trajectory_files = list(trajectory_path.glob("SQ*.csv"))
all_data = []
for file in trajectory_files:
    print(f"Reading {file.name}")
    df = pd.read_csv(file)
    df['flight_id'] = file.stem  # Add flight ID from filename
    all_data.append(df)
    
all_trajectories = pd.concat(all_data, ignore_index=True)
all_trajectories

In [37]:
c = ComplexityCalculator()
grid = c.create_4d_grid(all_trajectories)

In [ ]:
type(grid)

In [ ]:
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

In [ ]:
# Example usage:
def calculate_airspace_complexity(trajectory_files: List[str]) -> Dict:
    """Calculate airspace complexity from trajectory files"""
    # Read trajectories
    trajectories = []
    for file in trajectory_files:
        df = pd.read_csv(file)
        trajectories.append(df)
    
    # Initialize calculator
    calculator = ComplexityCalculator()
    
    # Create 4D grid and assign aircraft
    calculator.create_4d_grid(trajectories)
    
    # Calculate complexity metrics
    metrics = calculator.calculate_complexity_metrics()
    
    return metrics

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

class CellAnalyzer:
    def __init__(self, calculator):
        self.calculator = calculator

    def print_cell_summary(self):
        """Print summary statistics about all cells"""
        print(f"\nTotal number of cells: {len(self.calculator.cells)}")
        
        # Calculate occupancy stats
        aircraft_per_cell = [len(cell.aircraft_records) for cell in self.calculator.cells.values()]
        print("\nOccupancy Statistics:")
        print(f"Mean aircraft per cell: {np.mean(aircraft_per_cell):.2f}")
        print(f"Max aircraft per cell: {np.max(aircraft_per_cell)}")
        print(f"Cells with multiple aircraft: {sum(n > 1 for n in aircraft_per_cell)}")

    def print_cell_contents(self, num_cells=3):
        """Print detailed contents of first n cells"""
        print(f"\nDetailed contents of first {num_cells} cells:")
        
        for i, (cell_key, cell) in enumerate(self.calculator.cells.items()):
            if i >= num_cells:
                break
                
            print(f"\nCell {i+1} at coordinates {cell_key}:")
            print(f"Boundaries:")
            print(f"  Longitude: {cell.x_min:.2f} to {cell.x_max:.2f}")
            print(f"  Latitude: {cell.y_min:.2f} to {cell.y_max:.2f}")
            print(f"  Altitude: {cell.z_min:.0f} to {cell.z_max:.0f} ft")
            print(f"  Time: {cell.t_min} to {cell.t_max}")
            print(f"Number of aircraft: {len(cell.aircraft_records)}")
            
            # Show sample of aircraft in cell
            print("\nSample aircraft records:")
            for j, aircraft in enumerate(cell.aircraft_records[:2]):  # Show first 2 aircraft
                print(f"  Aircraft {j+1}:")
                for key, value in aircraft.items():
                    print(f"    {key}: {value}")

    def get_cell_statistics(self):
        """Get various statistics about the cells"""
        stats = {}
        
        # Count aircraft by altitude layer
        alt_counts = {}
        for cell_key, cell in self.calculator.cells.items():
            alt_layer = cell_key[2]  # z-coordinate
            if alt_layer not in alt_counts:
                alt_counts[alt_layer] = 0
            alt_counts[alt_layer] += len(cell.aircraft_records)
        
        stats['aircraft_by_altitude'] = alt_counts
        
        # Count aircraft by time period
        time_counts = {}
        for cell_key, cell in self.calculator.cells.items():
            time_period = cell_key[3]  # t-coordinate
            if time_period not in time_counts:
                time_counts[time_period] = 0
            time_counts[time_period] += len(cell.aircraft_records)
            
        stats['aircraft_by_time'] = time_counts
        
        return stats

    def find_busiest_cells(self, top_n=5):
        """Find the cells with the most aircraft"""
        cell_occupancy = {
            cell_key: len(cell.aircraft_records) 
            for cell_key, cell in self.calculator.cells.items()
        }
        
        busiest_cells = sorted(cell_occupancy.items(), 
                             key=lambda x: x[1], 
                             reverse=True)[:top_n]
        
        print(f"\nTop {top_n} busiest cells:")
        for cell_key, count in busiest_cells:
            cell = self.calculator.cells[cell_key]
            print(f"\nCell at coordinates {cell_key}:")
            print(f"Number of aircraft: {count}")
            print(f"Altitude range: {cell.z_min:.0f} to {cell.z_max:.0f} ft")
            
            # Count different vertical states
            states = {'climb': 0, 'cruise': 0, 'descent': 0}
            for aircraft in cell.aircraft_records:
                states[aircraft['vertical_state']] += 1
            print("Vertical states:", states)

def analyze_trajectories(file_paths):
    """Analyze multiple trajectory files"""
    # Read and combine all trajectory files
    all_data = []
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        df['flight_id'] = file_path.stem
        all_data.append(df)
    
    combined_data = pd.concat(all_data, ignore_index=True)
    
    # Create calculator and process data
    calculator = ComplexityCalculator()
    calculator.create_4d_grid(combined_data)
    
    # Create analyzer and show results
    analyzer = CellAnalyzer(calculator)
    
    # Print basic summary
    analyzer.print_cell_summary()
    
    # Show detailed contents of first few cells
    analyzer.print_cell_contents(num_cells=3)
    
    # Find busiest cells
    analyzer.find_busiest_cells(top_n=5)
    
    # Get statistics
    stats = analyzer.get_cell_statistics()
    
    # Print altitude distribution
    print("\nAircraft distribution by altitude layer:")
    for alt_layer, count in sorted(stats['aircraft_by_altitude'].items()):
        print(f"Layer {alt_layer}: {count} aircraft")
    
    # Print time distribution
    print("\nAircraft distribution by time period:")
    for time_period, count in sorted(stats['aircraft_by_time'].items()):
        print(f"Period {time_period}: {count} aircraft")

# Usage example:
if __name__ == "__main__":
    trajectory_files = list(Path("aircraft_trajectories").glob("SQ*.csv"))
    analyze_trajectories(trajectory_files)